In [ ]:
# 구글 드라이브 접근 코드
from google.colab import drive
drive.mount('/content/drive')
My_path = '/content/drive/MyDrive/LG_Academy/AI_Code/'

## 라이브러리 import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import precision_recall_curve, auc

In [ ]:
# 재현성을 위한 코드
random_seed = 777

tf.keras.utils.set_random_seed(random_seed)
# tf.random.normal([5], 0, 1)

## 데이터 불러오기

In [ ]:
DataLength = 2774
NoOfSensor = 3
NoOfData   = 180

DataLength, NoOfSensor

In [ ]:
NormalSet   = np.zeros([NoOfData, DataLength, NoOfSensor])
AbnormalSet = np.zeros([NoOfData, DataLength, NoOfSensor])

for i in range(NoOfData):
    tempNormalPath   = My_path + './SpotWeldingData/Normal_%d'  %(i+1)
    tempAbnormalPath = My_path + './SpotWeldingData/Abnormal_%d'%(i+1)

    tempNormal       = np.array(pd.read_csv(tempNormalPath  , header = None))
    tempAbnormal     = np.array(pd.read_csv(tempAbnormalPath, header = None))

    NormalSet[i, :, :]         = tempNormal
    AbnormalSet[i, :, :]      = tempAbnormal


NormalSet.shape, AbnormalSet.shape

In [ ]:
# 학습: 정상 데이터만 사용

TrainData = NormalSet[:-30, :, :]
ValidData = np.concatenate([NormalSet[-30:   , :, :], AbnormalSet[-30:   , :, :]], axis=0)

TrainData.shape, ValidData.shape

In [ ]:
TrainData = TrainData.reshape([TrainData.shape[0], TrainData.shape[1] * TrainData.shape[2]], order = 'F')
ValidData = ValidData.reshape([ValidData.shape[0], ValidData.shape[1] * ValidData.shape[2]], order = 'F')

TrainData.shape, ValidData.shape

## hyperparameter 설정

In [ ]:
learningRate = 0.0001
noOfNeuron   = 16
Epoch        = 200
latent_dim   = 2

## Keras 기반 AutoEncoder 구조(Architecture) 설계 - ANN

In [ ]:
def AutoEncoder(input_data, latent_dim):
    keras.backend.clear_session()
    tf.keras.utils.set_random_seed(random_seed)

    Encoder = keras.Sequential(name = 'Encoder')
    Encoder.add(keras.layers.InputLayer(shape = (input_data.shape[1],), name = 'Encoder_Input')) # Encoder Input layer
    Encoder.add(keras.layers.Dense(units = noOfNeuron, activation = keras.activations.relu,
                                   name = 'Encoder_Hidden1'))                                    # Encoder Hidden Layer 1
    Encoder.add(keras.layers.Dense(units = noOfNeuron, activation = keras.activations.relu,
                                   name = 'Encoder_Hidden2'))                                    # Encoder Hidden Layer 2
    Encoder.add(keras.layers.Dense(units = noOfNeuron, activation = keras.activations.relu,
                                   name = 'Encoder_Hidden3'))                                    # Encoder Hidden Layer 3
    Encoder.add(keras.layers.Dense(units = latent_dim, activation = keras.layers.LeakyReLU(),
                                   name = 'Encoder_Output' ))                                    # Encoder Output Layer(Hidden)


    Decoder = keras.Sequential(name = 'Decoder')
    Decoder.add(keras.layers.InputLayer(shape = (latent_dim,), name = 'Decoder_Input'))          # Decoder Input Layer
    Decoder.add(keras.layers.Dense(units = noOfNeuron, activation = keras.activations.relu,
                                   name = 'Decoder_Hidden1'))                                    # Decoder Hidden Layer 1
    Decoder.add(keras.layers.Dense(units = noOfNeuron, activation = keras.activations.relu,
                                   name = 'Decoder_Hidden2'))                                    # Decoder Hidden Layer 2
    Decoder.add(keras.layers.Dense(units = noOfNeuron, activation = keras.activations.relu,
                                   name = 'Decoder_Hidden3'))                                    # Decoder Hidden Layer 3
    Decoder.add(keras.layers.Dense(units = input_data.shape[1],
                                   name = 'Decoder_Output' ))                                    # Decoder Output Layer(Hidden)

    AutoEncoder = keras.Sequential([keras.layers.InputLayer(shape = (input_data.shape[1],)), Encoder, Decoder])

    AutoEncoder.compile(optimizer= keras.optimizers.Adam(learning_rate = learningRate),
                        loss=keras.losses.MAE)
    return AutoEncoder

## AutoEncoder 구조 확인

In [ ]:
AE_model = AutoEncoder(TrainData, latent_dim)

AE_model.summary()

In [ ]:
# Encoder 구조
encoder = AE_model.layers[0]
encoder.summary()

In [ ]:
# Decoder 구조
decoder = AE_model.layers[-1]
decoder.summary()

## Keras 기반 Autoencoder 학습 및 평가

In [ ]:
# 모델 학습
hist = AE_model.fit(TrainData, TrainData, epochs=Epoch, verbose = 1)

In [ ]:
# 모델 저장
AE_model.save(My_path + 'MLmodels/AE_model.keras')

In [ ]:
# 학습 과정 확인(Loss, Accuracy)
fig, loss_ax = plt.subplots()

loss_ax.plot(hist.history['loss'], label='train loss (mae)', color = "tab:red")
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

plt.show()

## Threshold 설정

In [ ]:
# # 모델 불러오기
# AE_model = keras.models.load_model(My_path + 'MLmodels/AE_model.keras')

In [ ]:
TrainData.shape

In [ ]:
# 학습 에러 정의
reconstructions   = AE_model.predict(TrainData, verbose = 0)
reconstructions   = reconstructions.reshape([-1, 2774, 3], order = 'F')
TrainData_reshape = TrainData.reshape([-1, 2774, 3], order = 'F')
Error           = abs((TrainData_reshape - reconstructions))

TrainData_reshape.shape, reconstructions.shape, Error.shape

In [ ]:
plt.figure(figsize = (12, 6))
plt.subplot(3, 1, 1)
plt.plot(TrainData_reshape[0, :, -1])
plt.grid(alpha = 0.4)
plt.title('Real Train (Normal)')

plt.subplot(3, 1, 2)
plt.plot(reconstructions[0, :, -1])
plt.grid(alpha = 0.4)
plt.title('Reconstruction')

plt.subplot(3, 1, 3)
plt.plot(Error[0, :, -1])
plt.grid(alpha = 0.4)
plt.title('Reconstruction Error')

plt.tight_layout()
plt.show()

In [ ]:
# 학습 데이터 복원에러(MAE) 정의
Train_MAE = np.mean(Error, axis = 1)
Train_MAE = np.mean(Train_MAE, axis = 1)

Train_MAE.shape

In [ ]:
# PDF 그래프
bins = np.arange(0, 0.3, 0.005)

plt.figure(figsize=(10,6))
plt.hist(Train_MAE, label = 'Train', edgecolor='k', bins = bins, alpha = 0.9, density = True, color = 'C0')

plt.grid(alpha = 0.8)
plt.xlabel('Reconstruction Error')
plt.ylabel('Density')
plt.legend(loc='upper right')
plt.title('PDF of Reconstruction')
plt.show()

In [ ]:
# 허용범위 약 90%
Threshold_1 = np.mean(Train_MAE) + np.std(Train_MAE)*1.65

# 허용범위 약 95%
Threshold_2 = np.mean(Train_MAE) + np.std(Train_MAE)*1.96

# 허용범위 약 99%
Threshold_3 = np.mean(Train_MAE) + np.std(Train_MAE)*2.56

Threshold_1, Threshold_2, Threshold_3

In [ ]:
# 학습 데이터 이상탐지 결과 (150개)

sum(Train_MAE <= Threshold_1), sum(Train_MAE <= Threshold_2), sum(Train_MAE <= Threshold_3)

In [ ]:
# 학습 데이터 이상탐지 결과
print(np.round(sum(Train_MAE <= Threshold_1)/Train_MAE.shape[0]*100, 2), '%', end=' ')
print(np.round(sum(Train_MAE <= Threshold_2)/Train_MAE.shape[0]*100, 2), '%', end=' ')
print(np.round(sum(Train_MAE <= Threshold_3)/Train_MAE.shape[0]*100, 2), '%', end=' ')

In [ ]:
# PDF 그래프
bins = np.arange(0, 0.3, 0.005)

plt.figure(figsize=(10,6))
plt.hist(Train_MAE, label = 'Train', edgecolor='k', bins = bins, alpha = 0.9, density = True, color = 'C0')

plt.vlines(x=Threshold_1, ymin = 0, ymax=45, label = '90%', linestyle = 'dashed', color = 'purple')
plt.vlines(x=Threshold_2, ymin = 0, ymax=45, label = '95%', linestyle = 'solid', color = 'purple')
plt.vlines(x=Threshold_3, ymin = 0, ymax=45, label = '99%', linestyle = 'dotted', color = 'purple')

plt.grid(alpha = 0.8)
plt.xlabel('Reconstruction Error')
plt.ylabel('Density')
plt.legend(loc='upper right')
plt.title('PDF of Reconstruction')
plt.show()

## 검증 데이터 이상탐지

In [ ]:
# 검증 데이터 에러 정의
reconstructions_Valid = AE_model.predict(ValidData, verbose = 0)
reconstructions_Valid = reconstructions_Valid.reshape([-1, 2774, 3], order = 'F')
ValidData_reshape = ValidData.reshape([-1, 2774, 3], order = 'F')
Error_Valid           = abs((ValidData_reshape - reconstructions_Valid))

ValidData_reshape.shape, reconstructions_Valid.shape, Error_Valid.shape

In [ ]:
# 검증 데이터 복원에러(MAE) 정의
Valid_MAE = np.mean(Error_Valid, axis = 1)
Valid_MAE = np.mean(Valid_MAE, axis = 1)
Valid_MAE.shape

In [ ]:
# 검증 데이터 복원에러 분할
NoOfValidData   = int(ValidData.shape[0]/2)

Valid_Normal   = Valid_MAE[:NoOfValidData]
Valid_Abnormal = Valid_MAE[NoOfValidData:]

Valid_Normal.shape, Valid_Abnormal.shape

In [ ]:
# 검증 데이터 이상탐지 결과(정상)

sum(Valid_Normal <= Threshold_1), sum(Valid_Normal <= Threshold_2), sum(Valid_Normal <= Threshold_3)

print(np.round(sum(Valid_Normal <= Threshold_1)/Valid_Normal.shape[0]*100, 2), '%', end=' ')
print(np.round(sum(Valid_Normal <= Threshold_2)/Valid_Normal.shape[0]*100, 2), '%', end=' ')
print(np.round(sum(Valid_Normal <= Threshold_3)/Valid_Normal.shape[0]*100, 2), '%', end=' ')

In [ ]:
# 검증 데이터 이상탐지 결과(고장)

sum(Valid_Abnormal >= Threshold_1), sum(Valid_Abnormal >= Threshold_2), sum(Valid_Abnormal >= Threshold_3)

print(np.round(sum(Valid_Abnormal >= Threshold_1)/Valid_Abnormal.shape[0]*100, 2), '%', end=' ')
print(np.round(sum(Valid_Abnormal >= Threshold_2)/Valid_Abnormal.shape[0]*100, 2), '%', end=' ')
print(np.round(sum(Valid_Abnormal >= Threshold_3)/Valid_Abnormal.shape[0]*100, 2), '%', end=' ')

In [ ]:
# PDF 그래프
bins = np.arange(0, 0.3, 0.005)

plt.figure(figsize=(10,6))
plt.hist(Train_MAE, label = 'Train', edgecolor='k', bins = bins, alpha = 0.9, density = True, color = 'C0')
plt.hist(Valid_Normal, label = 'Valid Normal', edgecolor='k', bins = bins, alpha = 0.9, density = True, color = 'C2')
plt.hist(Valid_Abnormal, label = 'Valid Abnormal', edgecolor='k', bins = bins, alpha = 0.9, density = True, color = 'C3')



plt.vlines(x=Threshold_1, ymin = 0, ymax=73, label = '90%', linestyle = 'dashed', color = 'purple')
plt.vlines(x=Threshold_2, ymin = 0, ymax=73, label = '95%', linestyle = 'solid', color = 'purple')
plt.vlines(x=Threshold_3, ymin = 0, ymax=73, label = '99%', linestyle = 'dotted', color = 'purple')

plt.grid(alpha = 0.8)
plt.xlabel('Reconstruction Error')
plt.ylabel('Density')
plt.legend(loc='upper right')
plt.title('PDF of Reconstruction')
plt.show()

## Precision-Recall curve

In [ ]:
# Threshold 조정을 통한 PR curve 확인

# Valid 데이터에 대한 실제 라벨 생성 (정상: 0, 고장: 1)
Valid_Labels = np.concatenate([np.zeros(Valid_Normal.shape[0]), np.ones(Valid_Abnormal.shape[0])])
Valid_Errors = np.concatenate([Valid_Normal, Valid_Abnormal])

# 정렬된 고유한 에러 값을 Threshold로 사용
sorted_errors = np.sort(np.unique(Valid_Errors))

# Precision과 Recall 값을 저장할 리스트
precision_list = []
recall_list = []

# 다양한 Threshold 값에 대해 Precision과 Recall 계산
for threshold in sorted_errors:
    # 예측: 에러가 Threshold보다 크면 이상 (1), 작거나 같으면 정상 (0)
    predictions = (Valid_Errors > threshold).astype(int)

    # True Positives (TP): 실제 이상 (1) 이고 예측 이상 (1)
    tp = np.sum((Valid_Labels == 1) & (predictions == 1))

    # False Positives (FP): 실제 정상 (0) 이고 예측 이상 (1)
    fp = np.sum((Valid_Labels == 0) & (predictions == 1))

    # False Negatives (FN): 실제 이상 (1) 이고 예측 정상 (0)
    fn = np.sum((Valid_Labels == 1) & (predictions == 0))

    # Precision 계산 (TP / (TP + FP)), 0으로 나누는 경우 처리
    precision = tp / (tp + fp) if (tp + fp) > 0 else 1.0

    # Recall 계산 (TP / (TP + FN)), 0으로 나누는 경우 처리
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0

    precision_list.append(precision)
    recall_list.append(recall)

# 마지막 점 추가 (Threshold가 가장 클 때, 모든 예측이 정상일 경우)
precision_list.append(1.0)
recall_list.append(0.0)

# Precision-Recall Curve 시각화
plt.figure(figsize=(8, 6))
plt.plot(recall_list, precision_list, label='Precision-Recall curve (manual)')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='lower left')
plt.grid(alpha=0.8)
plt.show()

# AUC 계산
pr_auc_manual = 0.0
for i in range(1, len(recall_list)):
    pr_auc_manual += 0.5 * (precision_list[i-1] + precision_list[i]) * (recall_list[i-1] - recall_list[i])

print(f"Manual AUC: {pr_auc_manual:.2f}")

In [ ]:
# scikit-learn 라이브러리 함수 활용

# Valid MAE에 대한 실제 라벨 생성 (정상: 0, 고장: 1)
Valid_Labels = np.concatenate([np.zeros(Valid_Normal.shape[0]), np.ones(Valid_Abnormal.shape[0])])
Valid_Errors = np.concatenate([Valid_Normal, Valid_Abnormal])

# Precision-Recall curve 계산
precision, recall, thresholds = precision_recall_curve(Valid_Labels, Valid_Errors)

# AUC 계산
pr_auc = auc(recall, precision)

# Precision-Recall Curve 시각화
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, label=f'Precision-Recall curve (AUC = {pr_auc:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='lower left')
plt.grid(alpha=0.8)
plt.show()